In [1]:
import findspark
findspark.init() 

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

# Check Spark version
print("Spark version:", spark.version)
spark

Spark version: 3.5.5


## DATASETS

In [2]:
df=spark.read.csv("Customer_table.csv",header=True,inferSchema=True,mode="DROPMALFORMED")
ds=spark.read.csv("Admin_data.csv",header=True,inferSchema=True,mode="DROPMALFORMED")


In [3]:
df.show(10)

+---------------+---------------+-----------+-------------------+-------------+-------+---------------+---------------+-------------+---------+------------+---------------+--------------+----------------------+---------+----------------+------------+--------------+----------------+--------+-----------+
|Pickup DateTime|  Drop DateTime|Driver Name|Driver Phone Number|Trip Distance|Trip ID|Pickup Location|  Drop Location|Trip Duration|Trip Fare|Vehicle Type|   Trip Purpose|Passenger Name|Passenger Phone Number|Scheduled|Passenger Rating|Driver Rated|Payment Method|Payment Discount|Net Fare|Trip Status|
+---------------+---------------+-----------+-------------------+-------------+-------+---------------+---------------+-------------+---------+------------+---------------+--------------+----------------------+---------+----------------+------------+--------------+----------------+--------+-----------+
| 1/1/2016 21:11| 1/1/2016 21:17|     Almire|         9298608912|           21| 318886| 

In [4]:
df.printSchema()

root
 |-- Pickup DateTime: string (nullable = true)
 |-- Drop DateTime: string (nullable = true)
 |-- Driver Name: string (nullable = true)
 |-- Driver Phone Number: long (nullable = true)
 |-- Trip Distance: integer (nullable = true)
 |-- Trip ID: integer (nullable = true)
 |-- Pickup Location: string (nullable = true)
 |-- Drop Location: string (nullable = true)
 |-- Trip Duration: double (nullable = true)
 |-- Trip Fare: double (nullable = true)
 |-- Vehicle Type: string (nullable = true)
 |-- Trip Purpose: string (nullable = true)
 |-- Passenger Name: string (nullable = true)
 |-- Passenger Phone Number: long (nullable = true)
 |-- Scheduled: string (nullable = true)
 |-- Passenger Rating: double (nullable = true)
 |-- Driver Rated: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Payment Discount: string (nullable = true)
 |-- Net Fare: double (nullable = true)
 |-- Trip Status: string (nullable = true)



In [5]:
ds.show()

+---------------+---------------+--------+----------+---+---------+-----------------+-----------+---------------+-----+-----------+-------------+---------------+-----+------+--------+--------+-----+-------------+---------+---------+---------+----------+----------+-----------+--------------+---------------+---------+----------+------------+--------+----------+---------+
|     Start_time|       End_time|    Name|    Mobile|Age|Pin-Codes|           Source|Vaccine_cus|    Destination|Miles|Est_Costing|Ride_category|        Purpose| temp|clouds|pressure|humidity| wind|accquire_vehi|free_vehi|Lattitute|Longitude|locationID|rating_cus|Riders_Name|Riders_contact|Trusted_Contact|Rating_RI|Vaccine_Ri|Payment_mode|Discount|Final_cost|   Status|
+---------------+---------------+--------+----------+---+---------+-----------------+-----------+---------------+-----+-----------+-------------+---------------+-----+------+--------+--------+-----+-------------+---------+---------+---------+----------+---

In [6]:
ds.printSchema()

root
 |-- Start_time: string (nullable = true)
 |-- End_time: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Mobile: long (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Pin-Codes: integer (nullable = true)
 |-- Source: string (nullable = true)
 |-- Vaccine_cus: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Miles: double (nullable = true)
 |-- Est_Costing: double (nullable = true)
 |-- Ride_category: string (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- temp: double (nullable = true)
 |-- clouds: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- wind: double (nullable = true)
 |-- accquire_vehi: integer (nullable = true)
 |-- free_vehi: string (nullable = true)
 |-- Lattitute: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- locationID: integer (nullable = true)
 |-- rating_cus: integer (nullable = true)
 |-- Riders_Name: string (null

# 1.How many no of customers take trip from same location.

In [7]:
from pyspark.sql.functions import count,col
df.groupBy("Pickup Location").agg(count("Passenger Name").alias("customer_count")).orderBy('customer_count',ascending=False).show()

+-----------------+--------------+
|  Pickup Location|customer_count|
+-----------------+--------------+
|      Fort Pierce|           108|
|          Midtown|            78|
|  West Palm Beach|            54|
|             Cary|            52|
|  Lower Manhattan|            26|
|     Midtown East|            26|
|Flatiron District|            26|
|      East Harlem|            26|
|    Hudson Square|            26|
|          Jamaica|            26|
|         New York|            26|
|         Elmhurst|            26|
+-----------------+--------------+



# 2.what is priority for each ride category from each location.

In [8]:
ds.groupBy("Source","Ride_category").agg(count("*").alias("Count of ride category")).orderBy(col("Source"),col("Count of ride category").desc()).show()

+-----------------+-------------+----------------------+
|           Source|Ride_category|Count of ride category|
+-----------------+-------------+----------------------+
|             Cary|        Prime|                    12|
|             Cary|         Auto|                    12|
|             Cary|    Uber-Mini|                    12|
|             Cary|   Uber-Micro|                     8|
|             Cary|         Bike|                     8|
|      East Harlem|         Bike|                     6|
|      East Harlem|   Uber-Micro|                     6|
|      East Harlem|    Uber-Mini|                     6|
|      East Harlem|         Auto|                     4|
|      East Harlem|        Prime|                     4|
|         Elmhurst|         Auto|                     6|
|         Elmhurst|   Uber-Micro|                     6|
|         Elmhurst|        Prime|                     5|
|         Elmhurst|         Bike|                     5|
|         Elmhurst|    Uber-Min

# 3.what are the longest locations of customer travelled.

In [9]:
df.select("Passenger Name","Trip Distance").orderBy("Trip Distance",ascending=False).show()

+--------------+-------------+
|Passenger Name|Trip Distance|
+--------------+-------------+
|      Darlleen|           80|
|         Price|           80|
|      Charlena|           80|
|      Virginie|           80|
|        Haskel|           80|
|         Jacky|           80|
|      Collette|           80|
|          Moss|           80|
|          Elsy|           79|
|           Pen|           79|
|         Daron|           79|
|          Elsy|           79|
|        Shaine|           79|
|         Sandy|           79|
|         Marta|           79|
|       Romonda|           79|
|          Dora|           78|
|        Damien|           78|
|       Charlot|           78|
|         Jacky|           78|
+--------------+-------------+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import max
max_distance = df.select(max("Trip Distance").alias("max_trip")).collect()[0]["max_trip"]

df.select("Passenger Name", "Trip Distance").where(col("Trip Distance") == max_distance) .show()


+--------------+-------------+
|Passenger Name|Trip Distance|
+--------------+-------------+
|         Price|           80|
|      Darlleen|           80|
|      Virginie|           80|
|      Charlena|           80|
|        Haskel|           80|
|         Jacky|           80|
|      Collette|           80|
|          Moss|           80|
+--------------+-------------+



# 4. Drivers who completed ride with non -vaccinated customers.

In [11]:
ds.select("Riders_Name").filter((col("Vaccine_cus")=="NO") & (col("Status")=="Arrived")).show()

+-----------+
|Riders_Name|
+-----------+
|     Mikkel|
|        Kip|
|      Cindi|
|    Yasmeen|
|       Burk|
|      Natty|
|       Kain|
|      Sandy|
|   Kimberly|
|    Carolus|
|       Dora|
|    Romonda|
|    Ellette|
|     Mikkel|
|       Abbi|
|     Pattin|
|     Haskel|
|      Cindi|
|    Yasmeen|
|       Burk|
+-----------+
only showing top 20 rows



# 5.How many vaccinated customers have travelled

In [12]:
ds.select("Name").filter((col("Vaccine_cus")=="YES")).count()

400

# 6. .Customers who completed ride with non -vaccinated Drivers

In [13]:
ds.select("Name").filter((col("Vaccine_Ri")=="NO") & (col("Status")=="Arrived")).show()

+--------+
|    Name|
+--------+
| Mathian|
|  Horten|
|   Boyce|
|    Berk|
|  Aubert|
| Stavros|
|  Norene|
|  Stefan|
| Cassius|
|   Chadd|
|  Joelle|
|   Abbey|
|  Kristo|
|   Lesli|
|    Marv|
| Sherwin|
|    Andi|
|   Adamo|
|Ernaline|
| Balduin|
+--------+
only showing top 20 rows



# 7.who is the customer completed highest no of rides.

In [25]:
a=df.filter(col("Trip Status")=="Arrived").groupBy("Passenger Name").agg(count("*").alias("TotalRides")).orderBy(col("TotalRides").desc())

In [26]:
maxx=a.select(max("TotalRides").alias("Total")).collect()[0]["Total"]
a.select("Passenger Name","TotalRides").filter(col("TotalRides")==maxx).show()

+--------------+----------+
|Passenger Name|TotalRides|
+--------------+----------+
|         Sandy|         4|
|      Loutitia|         4|
|      Virginie|         4|
|          Bent|         4|
|        Shaine|         4|
|       Yasmeen|         4|
|          Cher|         4|
|        Sybila|         4|
|       Chelsie|         4|
+--------------+----------+



# 8.who is the driver completed highest no of rides.

In [29]:
a=ds.filter(col("Status")=="Arrived").groupBy("Riders_name").agg(count("*").alias("Tno")).orderBy(col("Tno").desc())
maxx=a.select(max("Tno").alias("Total")).collect()[0]["Total"]
a.select("Riders_nAme","Tno").filter(col("Tno")==maxx).show()

+-----------+---+
|Riders_nAme|Tno|
+-----------+---+
|      Sandy|  4|
|   Loutitia|  4|
|   Virginie|  4|
|       Bent|  4|
|     Shaine|  4|
|    Yasmeen|  4|
|       Cher|  4|
|     Sybila|  4|
|    Chelsie|  4|
+-----------+---+



# 9.what are first 10 age groups which uses uber services mostly.

In [36]:
ds.select("Age").groupBy("Age").agg(count("*").alias("Total number")).orderBy(col("Total number").desc()).show(10)

+---+------------+
|Age|Total number|
+---+------------+
| 69|          16|
| 15|          15|
| 32|          15|
| 28|          12|
| 78|          11|
| 43|          11|
| 49|          11|
| 21|          11|
| 34|          10|
| 44|          10|
+---+------------+
only showing top 10 rows



## 10.what is the count of different destination locations from same start location and also completed ride

In [46]:
from pyspark.sql.functions import countDistinct

ds.filter(col("Status")=="Arrived").groupBy("source").agg(countDistinct(col("destination")).alias("Destination count")).orderBy(col("Destination count").desc()).show()

+-----------------+-----------------+
|           source|Destination count|
+-----------------+-----------------+
|      Fort Pierce|               31|
|  West Palm Beach|               23|
|          Midtown|               19|
|             Cary|               15|
|     Midtown East|               11|
|Flatiron District|                9|
|      East Harlem|                8|
|          Jamaica|                8|
|         New York|                8|
|         Elmhurst|                8|
|  Lower Manhattan|                7|
|    Hudson Square|                7|
+-----------------+-----------------+



# 11.what is the most expensive Drive.

In [56]:
ds.select("name","Riders_name","source","destination","Final_cost").orderBy(col("Final_cost").desc()).show(1)

+----+-----------+------+-----------+----------+
|name|Riders_name|source|destination|Final_cost|
+----+-----------+------+-----------+----------+
| Ely|     Aurlie|  Cary|       Cary|    5406.0|
+----+-----------+------+-----------+----------+
only showing top 1 row

